In [1]:
import numpy as np
import os
import json
from symmer.operators import PauliwordOp, QuantumState
from openfermionpyscf import run_pyscf
from openfermion import (
    MolecularData,
    get_fermion_operator,
    jordan_wigner,
    QubitOperator,
)

/Users/qianfeng/anaconda3/envs/ez_adiabatic/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# DMRG scratch  

In [7]:
H_dict = {'IIIIIIII': -331.6627801640259,
 'ZIIIIIII': 59.14285951244939,
 'IIIIZIII': 3.7005815842211165,
 'ZIIIZIII': 2.7020254935585193,
 'IIZZIZZI': 1.9635309597930315,
 'IZZZIZZI': 1.9635309597930315,
 'IZIZIIIZ': 1.732260512565561,
 'IIIZIIII': 1.732260512565561,
 'IIZIIIII': 1.5468861329893158,
 'IIZIIIIZ': 1.5468861329893158,
 'IIIIIZIZ': 1.4073777879209017,
 'IIIIIZII': 1.4073777879209017,
 'IIIIZIZI': 1.3404217351344991,
 'IIIIZIZZ': 1.3404217351344991,
 'ZZZZIZZI': 1.2108384691287002,
 'ZIZZIZZI': 1.2108384691287002,
 'ZZIZIIIZ': 1.1946603634944104,
 'ZIIZIIII': 1.1946603634944104,
 'ZIIIIZII': 1.137191563579775,
 'ZIIIIZIZ': 1.137191563579775,
 'ZIIIZIZZ': 1.1074837005945546,
 'ZIIIZIZI': 1.1074837005945546,
 'ZIZIIIIZ': 1.0011909144993059,
 'ZIZIIIII': 1.0011909144993059,
 'IIIIIIIZ': 0.35931159476541924,
 'IIZIZIZZ': 0.22940053426868903,
 'IIZIIZIZ': 0.22674915820215394,
 'IIIIZZZZ': 0.22207625340938322,
 'IZIZZIIZ': 0.21943218298175432,
 'IIIZZIII': 0.21943218298175432,
 'IZZZZZZI': 0.21601509950649592,
 'IIZZZZZI': 0.21601509950649592,
 'IIIIZZIZ': 0.21004926749237246,
 'IIIIZZII': 0.21004926749237246,
 'IIIIIIZZ': 0.20376338540543765,
 'IIIIIIZI': 0.20376338540543765,
 'IIZIZIII': 0.19899545818630215,
 'IIZIZIIZ': 0.19899545818630215,
 'IIZIZIZI': 0.19226048056278983,
 'IXIZIIIZ': -0.18424713415832522,
 'IYZYIZZZ': -0.18424713415832522,
 'IYIYIIIZ': 0.18424713415832522,
 'IXZZIZZI': 0.18424713415832522,
 'IIIIZZZI': 0.17490964925475866,
 'IIZIIZII': 0.17290816128135972,
 'ZXZZIZZI': 0.14980884928504118,
 'ZXIZIIIZ': -0.14980884928504118,
 'ZYIYIIIZ': 0.14980884928504118,
 'ZYZYIZZZ': -0.14980884928504118,
 'IZIIIIIZ': 0.12456282253094157,
 'IZIIIIII': 0.12241970176288293,
 'IZZZZZII': 0.1167237531103278,
 'IIZZZZIZ': 0.1167237531103278,
 'IIIZIZZZ': 0.11375825228913318,
 'IZIZIZZI': 0.11375825228913318,
 'IZZZIIZI': 0.11212728151896593,
 'IIZZIIZZ': 0.11212728151896593,
 'IZZIIZZI': 0.10967083607230381,
 'IZZIIZZZ': 0.10967083607230381,
 'IIIZIZIZ': 0.10850754894478629,
 'IZIZIZIZ': 0.10850754894478629,
 'IZIZZIZZ': 0.10292588625038213,
 'IIIZZIZZ': 0.10292588625038213,
 'IIZZIIIZ': 0.10109069984753483,
 'IZZZIIIZ': 0.10109069984753483,
 'IIZZZZII': 0.09947616362119364,
 'IZZZZZIZ': 0.09947616362119364,
 'IIZZIIZI': 0.09929471396012568,
 'IZZZIIZZ': 0.09929471396012568,
 'IZIZIZII': 0.0902840628368186,
 'IIIZIZII': 0.0902840628368186,
 'IIZZIIII': 0.08821500249094744,
 'IZZZIIII': 0.08821500249094744,
 'IIZIIZZI': 0.08772793533735668,
 'IIZIIZZZ': 0.08772793533735668,
 'IIIZZIZI': 0.087274913674766,
 'IZIZZIZI': 0.087274913674766,
 'IIIZIZZI': 0.08381227450623872,
 'IZIZIZZZ': 0.08381227450623872,
 'IIXIIXIZ': 0.053840996920794214,
 'IIYIIYIZ': 0.053840996920794214,
 'IIIIIXXZ': 0.04716660415462457,
 'IIIIZYYZ': 0.04716660415462457,
 'IIXIIIXZ': 0.03714005370589919,
 'IIYIZIYZ': 0.03714005370589919,
 'IIXZIZZZ': 0.029945977782894453,
 'IIXIIIIZ': -0.029945977782894453,
 'IZXZIZZI': 0.029945977782894453,
 'IZXIIIII': -0.029945977782894453,
 'IIZZYZYI': -0.022863721025023015,
 'IZIIXIXI': -0.022863721025023015,
 'IIIIXIXI': -0.022863721025023015,
 'IZZZYZYI': -0.022863721025023015,
 'IZIXIIIZ': -0.02194290073494714,
 'IZZXIZZZ': 0.02194290073494714,
 'IZIXIIII': -0.02194290073494714,
 'IZZXIZZI': 0.02194290073494714,
 'IYYZIIZX': 0.021463213095809414,
 'IXZIZXYY': -0.021463213095809414,
 'IXZIIYXY': 0.021463213095809414,
 'IYZZZYXX': 0.021463213095809414,
 'IYZZIXYX': -0.021463213095809414,
 'IYYIZIZX': 0.021463213095809414,
 'IYYIIZIX': -0.021463213095809414,
 'IYYZZZIX': -0.021463213095809414,
 'IIIXIXIZ': 0.01822348610796769,
 'IZIXIXIZ': 0.01822348610796769,
 'IZIYIYIZ': 0.01822348610796769,
 'IIIYIYIZ': 0.01822348610796769,
 'IIZZZZXZ': 0.017247589489134154,
 'IZIIIIXI': -0.017247589489134154,
 'IZZZZZXI': 0.017247589489134154,
 'IIIIIIXZ': -0.017247589489134154,
 'IIZXZIYY': 0.016122826323587348,
 'IZXIZYZY': -0.016122826323587348,
 'IIXZIYIY': 0.016122826323587348,
 'IIYZIXIY': -0.016122826323587348,
 'IIIXZZYY': -0.016122826323587348,
 'IZYIZXZY': 0.016122826323587348,
 'IIZYIIXY': -0.016122826323587348,
 'IIIYIZXY': 0.016122826323587348,
 'IZIXIIXZ': 0.01565097257561613,
 'IZIYZIYZ': 0.01565097257561613,
 'IIIYZIYZ': 0.01565097257561613,
 'IIIXIIXZ': 0.01565097257561613,
 'IIIZIYXY': -0.014640938019707818,
 'IIIZZXYY': 0.014640938019707818,
 'IIYXZIZY': -0.014640938019707818,
 'IIXYZIZY': 0.014640938019707818,
 'IIYXIZIY': 0.014640938019707818,
 'IIXYIZIY': -0.014640938019707818,
 'IZZIIYXY': 0.014640938019707818,
 'IZZIZXYY': -0.014640938019707818,
 'IXZIIZZI': 0.013859045913385803,
 'IXIXIIII': -0.013859045913385803,
 'IXIIIIIZ': -0.013859045913385803,
 'IXZXIZZI': 0.013859045913385803,
 'IXZZZZII': -0.013680140485373403,
 'IYZYZZII': 0.013680140485373403,
 'IXIZZIZZ': 0.013680140485373403,
 'IYIYZIZI': -0.013680140485373403,
 'IXIZZIIZ': -0.013625580921721765,
 'IYIYZIIZ': 0.013625580921721765,
 'IYZYZZZZ': -0.013625580921721765,
 'IXZZZZZI': 0.013625580921721765,
 'IZXIZZYY': 0.013019130649537268,
 'IZYIIZXY': -0.013019130649537268,
 'IIIXZYZY': 0.013019130649537268,
 'IIZXIYIY': -0.013019130649537268,
 'IIZYIXIY': 0.013019130649537268,
 'IIIYZXZY': -0.013019130649537268,
 'IIXZZIYY': -0.013019130649537268,
 'IIYZIIXY': 0.013019130649537268,
 'IYYZZYXY': -0.01294529784897434,
 'IXZIIZIX': 0.01294529784897434,
 'IYYIIYXY': -0.01294529784897434,
 'IYZZZZIY': -0.01294529784897434,
 'IZXXIIIZ': 0.012875697356587388,
 'IZYYIIIZ': 0.012875697356587388,
 'IIXXIIIZ': 0.012875697356587388,
 'IIYYIIIZ': 0.012875697356587388,
 'IIIIIXIZ': -0.01283256755884025,
 'IIZZIXZZ': 0.01283256755884025,
 'IZIIIXII': -0.01283256755884025,
 'IZZZIXZI': 0.01283256755884025,
 'IYIZIZYX': 0.011416770468494746,
 'IXYIZXZY': 0.011416770468494746,
 'IXXIZYZY': -0.011416770468494746,
 'IYXZIYZX': -0.011416770468494746,
 'IYZIZIYX': -0.011416770468494746,
 'IYIIZZYX': 0.011416770468494746,
 'IYYZIXZX': 0.011416770468494746,
 'IYZZIIYX': -0.011416770468494746,
 'IXYYIIIZ': 0.010841451228922843,
 'IXXXIIIZ': 0.010841451228922843,
 'IXXZIZZI': -0.010841451228922843,
 'IYXYIZZI': 0.010841451228922843,
 'IYYXIZZI': -0.010841451228922843,
 'IYYIIZZI': -0.010841451228922843,
 'IYYZIIII': 0.010841451228922843,
 'IXXIIIII': 0.010841451228922843,
 'IIIYYIXI': 0.010619097784979567,
 'IZIYYIXZ': 0.010619097784979567,
 'IZIXXIXZ': 0.010619097784979567,
 'IIIXXIXI': 0.010619097784979567,
 'IXZZIIIZ': 0.010116301840077255,
 'IYIYIZZI': 0.010116301840077255,
 'IYZYIIII': -0.010116301840077255,
 'IXIZIZZI': -0.010116301840077255,
 'IYYIZXYY': 0.010046442627314668,
 'IXZIZIZX': -0.010046442627314668,
 'IYZZIIZY': 0.010046442627314668,
 'IYYZIXYY': 0.010046442627314668,
 'IXIIXIXI': -0.009941525025486944,
 'IYZIXZYZ': 0.009941525025486944,
 'IYIZYIXZ': -0.009941525025486944,
 'IYZYYZYZ': 0.009941525025486944,
 'IXIXXIXZ': -0.009941525025486944,
 'IXIYYIXZ': -0.009941525025486944,
 'IYZXXZYZ': 0.009941525025486944,
 'IXZZYZYI': -0.009941525025486944,
 'IXIIIIXI': 0.008561816195267967,
 'IXIXIIXZ': 0.008561816195267967,
 'IYZYZZXI': 0.008561816195267967,
 'IYZXIZYI': -0.008561816195267967,
 'IXZZZZXI': -0.008561816195267967,
 'IYZIIZYI': -0.008561816195267967,
 'IYIZZIYI': 0.008561816195267967,
 'IXIYZIYZ': 0.008561816195267967,
 'IXYIIZXY': -0.008517915246835074,
 'IXXIZZYY': 0.008517915246835074,
 'IYZIIYIX': 0.008517915246835074,
 'IYZZZYIX': 0.008517915246835074,
 'IYIIZYZX': -0.008517915246835074,
 'IYIZIYZX': -0.008517915246835074,
 'IYXZIZYX': 0.008517915246835074,
 'IYYZZZXX': -0.008517915246835074,
 'IIIIXIZZ': -0.00798791851232929,
 'IIIIXIIZ': 0.00798791851232929,
 'IIIIXIZI': -0.00798791851232929,
 'IIIIXIII': 0.00798791851232929,
 'IYIYZIZZ': -0.005118324290105437,
 'IYZYZZIZ': 0.005118324290105437,
 'IXIZZIZI': 0.005118324290105437,
 'IXZZZZIZ': -0.005118324290105437,
 'IIIIXXXI': 0.004922217324928143,
 'IIIIYYXI': 0.004922217324928143,
 'IIIIXXXZ': 0.004922217324928143,
 'IIIIYYXZ': 0.004922217324928143,
 'IXZIIZZZ': 0.004532725799902782,
 'IXZXIZZZ': 0.004532725799902782,
 'IXIIIIII': -0.004532725799902782,
 'IXIXIIIZ': -0.004532725799902782,
 'IIXIXIXI': 0.0041656403961984395,
 'IIYIYIXI': 0.0041656403961984395,
 'IIYIYIXZ': 0.0041656403961984395,
 'IIXIXIXZ': 0.0041656403961984395,
 'IIZZIIIX': -0.0031036956740500797,
 'IIXXZYYX': 0.0031036956740500797,
 'IZIIZZZX': -0.0031036956740500797,
 'IIYYIXXX': 0.0031036956740500797,
 'IYXZIYYY': 0.002898855221659672,
 'IYXIZYYY': 0.002898855221659672,
 'IYIZIZZY': 0.002898855221659672,
 'IXIIZZZX': -0.002898855221659672,
 'IXIXIXIZ': -0.002292255652622805,
 'IYZIIYZI': 0.002292255652622805,
 'IXZZIXZI': 0.002292255652622805,
 'IXIIIXII': -0.002292255652622805,
 'IYIZIYII': -0.002292255652622805,
 'IYZXIYZI': 0.002292255652622805,
 'IYZYIXZI': -0.002292255652622805,
 'IXIYIYIZ': -0.002292255652622805,
 'IYZYIIZZ': 0.0021856635810229017,
 'IXZZIIZZ': -0.0021856635810229017,
 'IYIYIZIZ': -0.0021856635810229017,
 'IXIZIZII': 0.0021856635810229017,
 'IIYXIYXX': -0.0016218073701705499,
 'IIXYZXYX': -0.0016218073701705499,
 'IIIZZIZX': 0.0016218073701705499,
 'IZZIIZIX': 0.0016218073701705499,
 'IIXYIYXX': -0.0014818883038795298,
 'IZZIZIZX': 0.0014818883038795298,
 'IIYXZXYX': -0.0014818883038795298,
 'IIIZIZIX': 0.0014818883038795298,
 'IXIZIZZZ': 0.0007251493888455873,
 'IXZZIIII': -0.0007251493888455873,
 'IYZYIIIZ': 0.0007251493888455873,
 'IYIYIZZZ': -0.0007251493888455873,
 'IXZZIIZI': 0.00010659207159990329,
 'IYZYIIZI': -0.00010659207159990329,
 'IXIZIZIZ': -0.00010659207159990329,
 'IYIYIZII': 0.00010659207159990329}

In [9]:
H_matrix = PauliwordOp.from_dictionary(H_dict).to_sparse_matrix.todense()
H_pauli = PauliwordOp.from_dictionary(H_dict)

In [10]:
eigenvalues, eigenvectors = np.linalg.eigh(H_matrix)

In [11]:
ref_E = eigenvalues[0]
ref_E

np.float64(-394.3538432576691)

In [13]:
from quimb.experimental.operatorbuilder import SparseOperatorBuilder
builder = SparseOperatorBuilder()
for bitstring, coeff in H_dict.items():
    ops = bitstring.lower()
    #ops = bitstring
    term = [
        (op,i) 
        for i, op in enumerate(ops)
        if op != 'i' 
    ]
    #print(term)
    builder.add_term(coeff, *term)

H_mpo = builder.build_mpo()

In [14]:
state_pw,energy_pw = dmrg_solver(H_mpo, bond_dims=[10, 16, 20, 20, 20], cutoffs=1e-12)


NameError: name 'dmrg_solver' is not defined

# DMRG large molecule

In [2]:
## import file from the currrent dir
cwd = os.getcwd()+'/mol'
files = [f for f in os.listdir(cwd) if os.path.isdir(os.path.join(cwd, f))]
files

['H2O', 'LiH', 'H4_square', 'H3_triangle']

In [3]:
selected_file = files[3]
selected_file

'H3_triangle'

In [4]:
sub_dir = os.path.join(cwd, selected_file)
json_data = os.path.join(sub_dir, 'chem_data.json')


with open(json_data, 'r') as infile:
    data = json.load(infile)

In [6]:
basis = data['basis']
geometry = data['geometry']
hamiltonian = data['H']
hamiltonian_matrix = PauliwordOp.from_dictionary(hamiltonian).to_sparse_matrix.todense()
hamiltonian_matrix.shape

(4096, 4096)

In [ ]:
eigenvalues, eigenvectors = np.linalg.eigh(hamiltonian_matrix)
eigenvalues[0]


In [ ]:
bond_dims = [2] * 4 + [5] * 4#<-------change bond dims
noises = [1e-4] * 4 + [1e-5] * 4 + [0]
thrds = [1e-10] * 8


mol = gto.M(atom=geometry, basis=basis, symmetry=None, verbose=0)
mf = scf.RHF(mol).run(conv_tol=1E-14)
ncas, n_elec, spin, ecore, h1e, g2e, orb_sym = itg.get_rhf_integrals(mf,
    ncore=0, ncas=None, g2e_symm=8)

driver = DMRGDriver(scratch="./tmp", symm_type=SymmetryTypes.SZ, n_threads=4)
driver.initialize_system(n_sites=ncas, n_elec=n_elec, spin=spin, orb_sym=orb_sym)

mpo = driver.get_qc_mpo(h1e=h1e, g2e=g2e, ecore=ecore, iprint=0)

ket = driver.get_random_mps(tag="GS", bond_dim=20, nroots=1)
energy = driver.dmrg(mpo, ket, n_sweeps=20, bond_dims=bond_dims, noises=noises,
    thrds=thrds, iprint=0)
print('DMRG energy = %20.15f' % energy)
energy_100 = energy #<-------change name
csfs, coeffs = driver.get_csf_coefficients(ket, cutoff=0.0, iprint=0)
state_matrix = np.zeros((csfs.shape[0], mf.mol.nao*2), dtype=bool)
for idx, det in enumerate(csfs):
    alpha_occ_mask  = 2*np.where(det==1)[0]
    beta_occ_mask   = 2*np.where(det==2)[0] + 1
    double_occ_mask = np.hstack((2*np.where(det==3)[0], 2*np.where(det==3)[0]+1))

    state_matrix[idx, alpha_occ_mask] = True
    state_matrix[idx, beta_occ_mask] = True
    state_matrix[idx, double_occ_mask] = True
psi = QuantumState(state_matrix, coeffs).normalize#<-------change name
